In [30]:
import gepia

In [31]:
bp=gepia.survival()

In [32]:
bp.setParam('jittersize',0.5)

In [33]:
cancer_type = gepia.CANCERType

In [34]:
bp.setParam('dataset',['BLCA', 'BRCA'])

In [35]:
result=bp.query()

./CCR7_survival_kpKZE.pdf


In [36]:
from IPython.display import IFrame

In [37]:
IFrame(result,width=500,height=500)

In [38]:
sigs=[['CCR7','LEF1','TCF7','SELL'],['CX3CR1','FGFBP2','FCGR3A'],['HAVCR2','TIGIT','LAG3','PDCD1','CXCL13','LAYN']]


In [39]:
params=bp.params

for sig in sigs:
    bp.setParams(params)
    bp.setParam('signature',sig)
    bp.query()

./CCR7_survival_d0JhQ.pdf
./CX3CR1_survival_zF1OP.pdf
./HAVCR2_survival_NUBAq.pdf


In [ ]:
gepia.

AttributeError: module 'gepia' has no attribute 'get'